In [80]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn import metrics
from sklearn.utils import shuffle

from sklearn.cluster import DBSCAN

from sklearn.preprocessing import PolynomialFeatures
from sklearn.manifold import TSNE
#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import plotly.offline as py
import plotly.tools as tls
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

import matplotlib.pyplot as plt

import umap
from catboost import CatBoostClassifier, Pool

За основу беру DataSet MAGIC Gamma Telescope Data Set https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope

In [81]:
df = pd.read_csv("magic04.csv")
df.columns=['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']

In [82]:
df.loc[df['class'] == 'g', 'class'] = 1
df.loc[df['class'] == 'h', 'class'] = 0
df = shuffle(df) 

Пустых значений нет и это хорошо

In [83]:
#df_temp = df['fM3Trans_abs']
#df_temp =  df.loc[(df['fWidth'] == 0) & (df['class'] == 0), 'fWidth']

#fig = px.scatter(x=df_temp, marginal_x="histogram", 
                 #color=df['class']
#                )
#fig.show()

#fig = px.box(x=df_temp)
#fig.show()

Немного EDA

In [84]:
df['fWidth_null'] = 0; df.loc[df['fWidth'] == 0, 'fWidth_null'] = 1
df['fM3Long_abs']=np.abs(df['fM3Long'])
df['fM3Trans_abs']=np.abs(df['fM3Trans'])

In [85]:
df_y = pd.DataFrame(df['class'])
df.drop(['class'], axis='columns', inplace=True)

In [86]:
poly = PolynomialFeatures(degree=3)
real_columns = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'fM3Long_abs', 'fM3Trans_abs']

df_P_Features = pd.DataFrame(poly.fit_transform(df[real_columns]))
df_P_Features.columns=poly.get_feature_names(real_columns)
df_P_Features.drop(['1']+real_columns, axis='columns', inplace=True)
df = pd.concat([df, df_P_Features], sort=False, axis=1)

Скалируем

In [87]:
pt = PowerTransformer()
pt = StandardScaler()
df_scaled = pd.DataFrame(pt.fit_transform(df))
df_scaled.columns = df.columns

TSNE UMAP

In [90]:
df_scaled_TSNE = pd.DataFrame(TSNE(n_components=2, init='random', perplexity=50, n_iter=3000).fit_transform(df_scaled), columns=['x', 'y'])

In [91]:
fig = px.scatter(x=df_scaled_TSNE['x'], y=df_scaled_TSNE['y'])
fig.show()

In [70]:
reducer = umap.UMAP(n_neighbors=2)
df_scaled_UMAP = pd.DataFrame(reducer.fit_transform(df_scaled), columns=['x', 'y'])
fig = px.scatter(x=df_scaled_UMAP['x'], y=df_scaled_UMAP['y'])
fig.show()

C:\Users\QQ\anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning:

Graph is not fully connected, spectral embedding may not work as expected.



KeyboardInterrupt: 